In [2]:
import torch

In [3]:
from torch._subclasses.fake_tensor import FakeTensorMode

In [4]:
x = torch.empty(2, 2, device="cpu")
y = torch.empty(4, 2, 2, device="cpu")
with FakeTensorMode() as mode:
    xf = mode.from_tensor(x)
    yf = mode.from_tensor(y)
    print(xf+yf)

FakeTensor(..., size=(4, 2, 2))


In [5]:
torch._C._dispatch_find_schema_or_throw("aten::sin", "")

In [13]:
from torch.fx.experimental.proxy_tensor import make_fx
# torch/testing/_internal/two_tensor.py
# A simple tensor subclass that holds two tensors internally, and runs every op on both tensors.
class TwoTensor(torch.Tensor):
    @staticmethod
    def __new__(cls, a, b):
        assert (
            a.device == b.device
            and a.layout == b.layout
            and a.requires_grad == b.requires_grad
            and a.dtype == b.dtype
        )
        # I guess it would be more accurate to represent the shape as torch.cat(a, b).shape
        shape = a.shape
        kwargs = {}
        kwargs["strides"] = a.stride()
        kwargs["storage_offset"] = a.storage_offset()
        kwargs["device"] = a.device
        kwargs["layout"] = a.layout
        kwargs["requires_grad"] = a.requires_grad
        kwargs["dtype"] = a.dtype
        out = torch.Tensor._make_wrapper_subclass(cls, shape, **kwargs)

        assert a.shape == b.shape
        assert a.stride() == b.stride()
        assert a.storage_offset() == b.storage_offset()
        return out

    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __repr__(self):
        a_repr = repr(self.a)
        b_repr = repr(self.b)
        return f"TwoTensor({a_repr}, {b_repr})"

    def __tensor_flatten__(self):
        return ["a", "b"], None

    @staticmethod
    def __tensor_unflatten__(inner_tensors, meta, outer_size, outer_stride):
        assert meta is None
        a, b = inner_tensors["a"], inner_tensors["b"]
        return TwoTensor(a, b)

    @classmethod
    def __torch_dispatch__(cls, func, types, args, kwargs):
        print("in python dispather?????")
        if kwargs is None:
            kwargs = {}
        args_a = pytree.tree_map_only(TwoTensor, lambda x: x.a, args)
        args_b = pytree.tree_map_only(TwoTensor, lambda x: x.b, args)

        kwargs_a = pytree.tree_map_only(TwoTensor, lambda x: x.a, kwargs)
        kwargs_b = pytree.tree_map_only(TwoTensor, lambda x: x.b, kwargs)

        out_a = func(*args_a, **kwargs_a)
        out_b = func(*args_b, **kwargs_b)
        assert type(out_a) == type(out_b)
        out_a_flat, spec = pytree.tree_flatten(out_a)
        out_b_flat = pytree.tree_leaves(out_b)
        # for aten ops that return non-tensors, just assume that
        # our two inner tensors return the same value
        out_flat = [
            TwoTensor(o_a, o_b) if isinstance(o_a, torch.Tensor) else o_a
            for o_a, o_b in zip(out_a_flat, out_b_flat)
        ]
        out = pytree.tree_unflatten(out_flat, spec)
        return return_and_correct_aliasing(func, args, kwargs, out)
        
def test_make_fx_with_subclass() -> None:
    def f(x, y):
        # Returns (TwoTensor, Tensor)
        return x * y, y + y
    x_a = torch.zeros(4)
    x_b = torch.zeros(4)
    y = torch.ones(4)

    # make_fx() is not responsible for unwrapping tensor subclass inputs,
    # so we do it manually here.
    # Why? In general, make_fx(f)(*args) promises that the graph returned has the same calling
    # convention as f(*args). Unwrapping tensor subclass inputs can potentially change
    # the number of input args to the graph, breaking that assumption
    def f_to_trace(x_a, x_b, y):
        x = TwoTensor(x_a, x_b)
        out1, out2 = f(x, y)
        out1_unwrapped_attrs, _ = out1.__tensor_flatten__()
        return (*[getattr(out1, attr) for attr in out1_unwrapped_attrs], out2)
    f_to_trace(x_a, x_b, y)
#     fx_g = make_fx(f_to_trace, tracing_mode='fake')(x_a, x_b, y)
#     self.assertExpectedInline(fx_g.code, """\



# def forward(self, x_a_1, x_b_1, y_1):
#     mul = torch.ops.aten.mul.Tensor(x_a_1, y_1);  x_a_1 = None
#     mul_1 = torch.ops.aten.mul.Tensor(x_b_1, y_1);  x_b_1 = None
#     add = torch.ops.aten.add.Tensor(y_1, y_1);  y_1 = None
#     return (mul, mul_1, add)
#     """)
torch._C._DisablePythonDispatcher()
test_make_fx_with_subclass()

in python dispather?????


NameError: name 'pytree' is not defined